## 오픈소스 프로젝트

In [2]:
# 모듈 import, 필요에 따라 더 추가
import pandas as pd
import os

#### 데이터셋 불러오기
###### trafficDay=서울시 일일 버스 이용량 데이터
###### weather=서울시 강수 정보

In [52]:
forders_time=os.listdir("서울특별시 공공자전거 이용정보(시간대별)_2021년")
forders_month=os.listdir("서울특별시 공공자전거 이용정보(월별)_2021년")
df_time=pd.DataFrame()
df_month=pd.DataFrame()
df_location=pd.read_csv("공공자전거 대여소 정보(21.12월 기준).csv",encoding='cp949')
df_member=pd.read_csv("서울특별시 공공자전거 신규가입자 정보(일별).csv",encoding='cp949')
'''
이 부분 오류가 난다.
이유 : csv파일 내에 데이터들 외적으로 텍스트 박스같은 것이 삽입 되어있을 수 있다.
이를 확인해보고, 삭제한 후 다시 csv파일로 바꾸어 저장하면 해결될 것 같다.
기본적으로 encoding방법은 cp949이다.

for i in range(len(forders_time)):
    file="서울특별시 공공자전거 이용정보(시간대별)_2021년/"+forders_time[i]
    df=pd.read_csv(file,encoding="cp949")
    df_time=pd.concat([df_time,df])
'''

for i in range(len(forders_month)):
    file="서울특별시 공공자전거 이용정보(월별)_2021년/"+forders_month[i]
    df=pd.read_csv(file,encoding="cp949")
    df_month=pd.concat([df_month,df])

#### 데이터 셋 확인
###### 어떤 데이터들이 있는지 확인해보기 위해 맨 위의 5개의 행을 불러온다.

In [26]:
df_month.head()

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분),이용거리(M),이용시간(본)
0,2021-01,3,중랑센터,일일(회원),M,AGE_003,2,47.68,0.40,1720.00,25.0,NaN,NaN
1,2021-01,3,중랑센터,정기,M,AGE_003,11,578.98,2.87,12390.48,166.0,NaN,NaN
2,2021-01,3,중랑센터,정기,M,AGE_004,1,81.65,0.53,2291.05,13.0,NaN,NaN
3,2021-01,5,상암센터 정비실,일일(회원),\N,AGE_004,1,511.95,3.53,15209.58,47.0,NaN,NaN
4,2021-01,5,상암센터 정비실,일일(회원),M,AGE_004,6,371.10,3.39,14642.60,134.0,NaN,NaN


In [53]:
df_location.head(20)  #None값이 많이 존재(NaN값이 많이 존재해서 이상해서, 20개 띄웠다.)

,대여소\n번호,보관소(대여소)명,소재지(위치),Unnamed: 3,Unnamed: 4,Unnamed: 5,설치\n시기,설치형태,Unnamed: 8,운영\n방식
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCD,QR,NaN
1,NaN,NaN,자치구,상세주소,위도,경도,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,거치\n대수,거치\n대수,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,102.0,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.5556488,126.9106293,2015-09-06,20,NaN,LCD
5,103.0,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.55495071,126.9108353,2015-09-06,14,NaN,LCD
6,104.0,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.55062866,126.9149857,2015-09-06,13,NaN,LCD
7,105.0,합정역 5번출구 앞,마포구,서울특별시 마포구 양화로 48,37.55000687,126.9148254,2015-09-06,5,NaN,LCD
8,106.0,합정역 7번출구 앞,마포구,서울특별시 마포구 독막로 4,37.54864502,126.9128265,2015-09-06,10,NaN,LCD
9,107.0,신한은행 서교동금융센터점 앞,마포구,서울특별시 마포구 월드컵북로 35,37.55751038,126.9185028,2015-09-06,5,NaN,LCD


In [30]:
df_member.head()

,가입일자,사용자코드,연령대코드,성별,가입 수
0,2018-07-20,회원-내국인,10대,NaN,221
1,2018-07-20,회원-내국인,20대,NaN,1189
2,2018-07-20,회원-내국인,30대,NaN,348
3,2018-07-20,회원-내국인,40대,NaN,201
4,2018-07-20,회원-내국인,50대,NaN,91


### 결측치 처리
###### 위의 데이터들을 확인한 후 결측치를 처리한다.

In [50]:
# 결측치 처리(1)

In [51]:
# 결측치 처리 (2)

### 이동거리/평균사용시간이 높은 대여소는?

In [38]:
# 이동거리/평균사용시간 (1)

In [39]:
# 이동거리/ 평균사용시간 (2)

### 운동량과 이동거리의 상관관계

In [40]:
# 운동량과 이동거리의 상관관계 (1)

In [41]:
# 운동량과 이동거리의 상관관계 (2)

### 따릉이를 가장 많이 이용하는 성별/연령대

In [42]:
# 따릉이를 가장 많이 이용하는 성별/연령대 (1)

In [43]:
# 따릉이를 가장 많이 이용하는 성별/연령대 (2)

### 서울시 구별 설치 거치대의 개수

In [44]:
# 서울시 구별 설치 거치대의 개수 (1)

In [45]:
# 서울시 구별 설치 거치대의 개수 (2)

### 서울시 구별로 거치대 개수와 대여건수

In [46]:
# 서울시 구별로 거치대 개수와 대여건수 (1)

In [47]:
# 서울시 구별로 거치대 개수와 대여건수 (2)

### 신규가입인원 대비 실제 이용자수

In [48]:
# 신규가입인원 대비 실제 이용자수 (1)

In [49]:
# 신규가입인원 대비 실제 이용자수 (2)